In [1]:
import pandas as pd
import numpy as np

# ranking and similarity algorithms 


In [2]:
df = pd.read_csv("dataset.csv")

df.head()

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,...,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,...,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,...,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,...,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,...,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic


In [3]:
null_columns = df.columns[df.isnull().any()]
print(null_columns)

Index(['artists', 'album_name', 'track_name'], dtype='object')


In [4]:
has_duplicates = df.duplicated().any()
print(f"Does the DataFrame have duplicates? {has_duplicates}")

Does the DataFrame have duplicates? False


In [5]:
#  checking for duplicates and na values
df.drop_duplicates()
df.dropna()

# now 0 duplicates and null vals
print(df.duplicated().sum())
print(df.isnull().sum())


print(df.shape)

0
Unnamed: 0          0
track_id            0
artists             1
album_name          1
track_name          1
popularity          0
duration_ms         0
explicit            0
danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
time_signature      0
track_genre         0
dtype: int64
(114000, 21)


In [6]:
# select features for useful data

# print(df.columns)

df.describe()

selected_feautures = ['danceability', 'energy', 'valence', 'tempo', 'instrumentalness', 'acousticness', 'speechiness', 'popularity']




danceability → Higher values mean the song is more danceable.
energy → Measures intensity. High energy = exciting, low energy = calm.
valence → Measures happiness. High valence = positive/happy mood, low valence = sad/serious.
tempo → Faster tempos often mean energetic/hype, slower tempos = chill/sad.
instrumentalness → Higher values mean fewer lyrics; often used for ambient or study music.
acousticness → Higher values mean more acoustic elements (often calmer, folk-like).
speechiness → High values indicate more spoken words (e.g., rap, podcasts).
popularity → Not directly a mood feature but can help weight recommendations.

In [7]:

# Mood	Danceability	Energy	Valence	Tempo	Instrumentalness	Acousticness	Speechiness
# Happy	High	Medium/High	High	Medium/High	Low	Low	Low
# Chill	Medium	Low	Medium	Low	Medium/High	High	Low
# Sad	Low	Low	Low	Low	Medium	High	Low
# Hype	High	High	Medium/High	High	Low	Low	Medium
# Focus	Low/Medium	Low	Medium	Low/Medium	High	Medium	Low

In [8]:
# need to make a column that dictates mood

# point system 1 - 3 3 high 2 medium 1 low 
def scoring(x):
    score = 0
    # Danceability
    if x['danceability'] > 0.7:
        score += 3
    elif 0.4 <= x['danceability'] <= 0.7:
        score += 2
    else:
        score += 1
    
    # Energy
    if x['energy'] > 0.8:
        score += 3
    elif 0.5 <= x['energy'] <= 0.8:
        score += 2
    else:
        score += 1
    
    # Valence
    if x['valence'] > 0.7:
        score += 3
    elif 0.4 <= x['valence'] <= 0.7:
        score += 2
    else:
        score += 1

    # Tempo
    if x['tempo'] > 120:
        score += 3
    elif 80 <= x['tempo'] <= 120:
        score += 2
    else:
        score += 1
    
    # Instrumentalness
    if x['instrumentalness'] < 0.2:
        score += 3
    elif 0.2 <= x['instrumentalness'] <= 0.5:
        score += 2
    else:
        score += 1
    
    # Acousticness
    if x['acousticness'] < 0.3:
        score += 3
    elif 0.3 <= x['acousticness'] <= 0.6:
        score += 2
    else:
        score += 1
    
    # Speechiness
    if x['speechiness'] < 0.3:
        score += 3
    elif 0.3 <= x['speechiness'] <= 0.6:
        score += 2
    else:
        score += 1
    
    # Popularity
    if x['popularity'] > 70:
        score += 3
    elif 40 <= x['popularity'] <= 70:
        score += 2
    else:
        score += 1
    
    return score

    

    



In [9]:
# score logic now 

score = df[selected_feautures].apply(scoring, axis=1)
print(score)




0         20
1         14
2         16
3         16
4         17
          ..
113995    12
113996    11
113997    17
113998    19
113999    15
Length: 114000, dtype: int64


In [10]:
# will now give you the mood based on emotions
def get_mood(x):
    mood = ""
    if 18 <= x <= 21:
        mood = "Happy"
    elif 15 <= x <= 18:
        mood = "Chill"
    elif 8 <= x <= 12:
        mood = "Sad"
    elif 22 <= x <= 24:
        mood = "Hype"
    elif 13 <= x <= 14:
        mood = "Focus"
    else:
        mood = "Unknown"
    return mood


In [11]:
# create both features one on score and one on mood

df['Score'] = df[selected_feautures].apply(scoring, axis=1)

df.head()

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre,Score
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,...,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic,20
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,...,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic,14
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,...,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic,16
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,...,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic,16
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,...,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic,17


In [12]:
df['mood'] = df['Score'].apply(get_mood)

df.head()

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre,Score,mood
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,...,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic,20,Happy
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,...,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic,14,Focus
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,...,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic,16,Chill
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,...,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic,16,Chill
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,...,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic,17,Chill


In [31]:
#create a filter mechanism

moods = ["Happy", "Sad", "Chill", "Hype", "Focus"]
user_mood = input("Please select what mood you are in, must type as shown: Happy, Sad, Chill, Hype, Focus")

while user_mood not in moods:
    print("not in the list of moods provided")
    user_mood = input("Please select what mood you are in, must type as shown: Happy, Sad, Chill, Hype, Focus")



In [32]:
filtered_df = df[df["mood"] == user_mood]

filtered_df.head()

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre,Score,mood
5002,5002,3khEEPRyBeOUabbmOPJzAG,Kenshi Yonezu,KICK BACK,KICK BACK,83,193495,False,0.577,0.941,...,0.1050,0.002070,0.000003,0.0891,0.292,101.921,4,anime,20,Happy
5022,5022,5elW2CKSoqjYoJ32AGDxf1,ENHYPEN,DIMENSION : ANSWER,Polaroid Love,78,184601,False,0.738,0.670,...,0.0439,0.458000,0.000000,0.3610,0.651,138.055,4,anime,21,Happy
5021,5021,7v8wKvNQQIxkugCFFjrkaO,Aimer,残響散歌,残響散歌,77,184893,False,0.367,0.896,...,0.0910,0.000627,0.000000,0.2070,0.380,170.863,4,anime,20,Happy
5007,5007,0UzymivvUH5s8z4PeWZJaK,ENHYPEN,BORDER : CARNIVAL,FEVER,77,172106,False,0.600,0.713,...,0.0592,0.064200,0.000000,0.1420,0.471,72.999,4,anime,19,Happy
5005,5005,6y4GYuZszeXNOXuBFsJlos,Eve,Kaikai Kitan / Ao No Waltz,Kaikai Kitan,75,221426,False,0.584,0.911,...,0.0900,0.011900,0.000000,0.1190,0.606,92.502,4,anime,21,Happy


In [33]:
# now will filter by genre of the music the user likes
genres = [
    'acoustic', 'afrobeat', 'alt-rock', 'alternative', 'ambient', 'anime', 
    'black-metal', 'bluegrass', 'blues', 'brazil', 'breakbeat', 'british', 
    'cantopop', 'chicago-house', 'children', 'chill', 'classical', 'club', 
    'comedy', 'country', 'dance', 'dancehall', 'death-metal', 'deep-house', 
    'detroit-techno', 'disco', 'disney', 'drum-and-bass', 'dub', 'dubstep', 
    'edm', 'electro', 'electronic', 'emo', 'folk', 'forro', 'french', 'funk', 
    'garage', 'german', 'gospel', 'goth', 'grindcore', 'groove', 'grunge', 
    'guitar', 'happy', 'hard-rock', 'hardcore', 'hardstyle', 'heavy-metal', 
    'hip-hop', 'honky-tonk', 'house', 'idm', 'indian', 'indie-pop', 'indie', 
    'industrial', 'iranian', 'j-dance', 'j-idol', 'j-pop', 'j-rock', 'jazz', 
    'k-pop', 'kids', 'latin', 'latino', 'malay', 'mandopop', 'metal', 
    'metalcore', 'minimal-techno', 'mpb', 'new-age', 'opera', 'pagode', 
    'party', 'piano', 'pop-film', 'pop', 'power-pop', 'progressive-house', 
    'psych-rock', 'punk-rock', 'punk', 'r-n-b', 'reggae', 'reggaeton', 
    'rock-n-roll', 'rock', 'rockabilly', 'romance', 'sad', 'salsa', 'samba', 
    'sertanejo', 'show-tunes', 'singer-songwriter', 'ska', 'sleep', 
    'songwriter', 'soul', 'spanish', 'study', 'swedish', 'synth-pop', 
    'tango', 'techno', 'trance', 'trip-hop', 'turkish', 'world-music'
]

# Function to get user input for genres and handle edge cases

# rewrite this function to understand it ur self 
def get_user_genres():
    print("Available genres:")
    print(", ".join(genres))
    print("You can select multiple genres separated by commas.")
    print("Type 'exit' to quit genre selection.")
    while True:
        user_input = input("Please select the genres of music you like: ")
        if user_input.lower() == 'exit':
            print("Exiting genre selection.")
            return []
        # Split input into a list, strip whitespace, and convert to lowercase
        user_genres = [genre.strip().lower() for genre in user_input.split(",")]
        # Check if all selected genres are valid
        invalid_genres = [genre for genre in user_genres if genre not in genres]
        if invalid_genres:
            print(f"Invalid genres: {', '.join(invalid_genres)}")
            print("Please select only from the available genres.")
        else:
            return user_genres
        


In [34]:
# implement a filter based on the music the people enjoy. From there we can soon start mixing music

user_genres = get_user_genres()

df = filtered_df[filtered_df['track_genre'].isin(user_genres)]


df.shape

Available genres:
acoustic, afrobeat, alt-rock, alternative, ambient, anime, black-metal, bluegrass, blues, brazil, breakbeat, british, cantopop, chicago-house, children, chill, classical, club, comedy, country, dance, dancehall, death-metal, deep-house, detroit-techno, disco, disney, drum-and-bass, dub, dubstep, edm, electro, electronic, emo, folk, forro, french, funk, garage, german, gospel, goth, grindcore, groove, grunge, guitar, happy, hard-rock, hardcore, hardstyle, heavy-metal, hip-hop, honky-tonk, house, idm, indian, indie-pop, indie, industrial, iranian, j-dance, j-idol, j-pop, j-rock, jazz, k-pop, kids, latin, latino, malay, mandopop, metal, metalcore, minimal-techno, mpb, new-age, opera, pagode, party, piano, pop-film, pop, power-pop, progressive-house, psych-rock, punk-rock, punk, r-n-b, reggae, reggaeton, rock-n-roll, rock, rockabilly, romance, sad, salsa, samba, sertanejo, show-tunes, singer-songwriter, ska, sleep, songwriter, soul, spanish, study, swedish, synth-pop, tan

(560, 23)

In [35]:
df = df.sort_values(by=['popularity', 'Score'], ascending=[False, False])


df.head()

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre,Score,mood
5002,5002,3khEEPRyBeOUabbmOPJzAG,Kenshi Yonezu,KICK BACK,KICK BACK,83,193495,False,0.577,0.941,...,0.1050,0.002070,0.000003,0.0891,0.292,101.921,4,anime,20,Happy
5022,5022,5elW2CKSoqjYoJ32AGDxf1,ENHYPEN,DIMENSION : ANSWER,Polaroid Love,78,184601,False,0.738,0.670,...,0.0439,0.458000,0.000000,0.3610,0.651,138.055,4,anime,21,Happy
5021,5021,7v8wKvNQQIxkugCFFjrkaO,Aimer,残響散歌,残響散歌,77,184893,False,0.367,0.896,...,0.0910,0.000627,0.000000,0.2070,0.380,170.863,4,anime,20,Happy
5007,5007,0UzymivvUH5s8z4PeWZJaK,ENHYPEN,BORDER : CARNIVAL,FEVER,77,172106,False,0.600,0.713,...,0.0592,0.064200,0.000000,0.1420,0.471,72.999,4,anime,19,Happy
5005,5005,6y4GYuZszeXNOXuBFsJlos,Eve,Kaikai Kitan / Ao No Waltz,Kaikai Kitan,75,221426,False,0.584,0.911,...,0.0900,0.011900,0.000000,0.1190,0.606,92.502,4,anime,21,Happy


In [39]:
user_songs  = input("how many songs would you like recommended: ")

top_5_songs_list = df['track_name'].head(int(user_songs)).tolist()

print(top_5_songs_list)

['KICK BACK', 'Polaroid Love', '残響散歌', 'FEVER', 'Kaikai Kitan', '紅蓮華', 'unravel', 'シルエット', '115万キロのフィルム', '点描の唄', 'ピースサイン', 'Given-Taken', '4:00A.M.', 'One In A Billion', 'crossing field', 'Shinzo wo Sasageyo!', 'TFW (That Feeling When)', 'インフェルノ', 'Black Catcher', 'インフェルノ', '花の塔', 'Sign', '炎', 'カタオモイ', "il vento d'oro", 'Cry Baby', 'Black Rover', 'Black Catcher', 'Let Me In (20 CUBE)', 'Not For Sale', 'FLY HIGH!!', 'JoJo Pose', 'Guren no Yumiya', '感電', '春愁', 'ポラリス', 'SIU', 'Just A Little Bit', 'First Love', 'Naruto Bluebird Freestyle', 'フィクション', 'Upper Side Dreamin’', '遥か彼方', 'ホタルノヒカリ', 'She Bad', 'Hasty', 'Cry Baby', 'Harukamirai', 'Brave Shine', 'ALIVE', 'Flicker', 'Touch off', 'ADAMAS', 'asphyxia', 'Bunny Thot', '死神', 'Bars No Jutsu', 'COLORS', 'Kokoronashi', '虹', '奏(かなで)', 'Make the change', '命に嫌われている。', "I'm Toxic", 'Hey Tayo - Tayo Opening Theme Song', 'Attention, please!', "Let's Ride", 'Mind brand', 'THE HERO !! 〜怒れる拳に火をつけろ〜', 'ANIMA', '空に歌えば', 'Ghost Rule', 'Shinzo wo Sasagey

In [ ]:
# need to have a simple react front end that takes in all this info, and then a way to mix all these songs